<img src='https://www.dropbox.com/s/125k1wnxkl6bgs0/class_logo.jpg?dl=1' alt="Smiley face" align="center" style="width: 600px;"/>


# Notebook 5




### Topics covered
* * *
 * Portfolio optimization with constraints
 * Estimation uncertainty
 * Statistical tests
 * Minimum-Variance investing
 * Risk-parity investing
 * Portfolio choice using equilibirum thinking


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#GlobalFinMonthly
url="https://www.dropbox.com/s/3k35mt3t57ygff2/GlobalFinMonthly.csv?dl=1"
Data = pd.read_csv(url,na_values=-99)
# tell python Date is date:
Data['Date']=pd.to_datetime(Data['Date'])
# set an an index
Data=Data.set_index(['Date'])
Data=Data.rename(columns={Data.columns[1]: "MKTUS",Data.columns[2]: "BondUS",
                          Data.columns[3]: "EM",Data.columns[4]: "MKTxUS",Data.columns[5]: "BondxUS" })
Re=(Data.drop('RF',axis=1)).subtract(Data['RF'],axis='index')
Re.tail()

MKTUS    BondUS        EM    MKTxUS   BondxUS
Date                                                      
2016-08-31  0.0050 -0.008617  0.024986  0.000638 -0.009752
2016-09-30  0.0025 -0.016617  0.012953  0.012536  0.009779
2016-10-31 -0.0202 -0.049660  0.002274 -0.020583 -0.043676
2016-11-30  0.0486 -0.081736 -0.046071 -0.019898 -0.050459
2016-12-31  0.0182 -0.005596  0.002604  0.034083 -0.023507

# Practical approaches to deal with Estimation Uncertainty




* Never apply a Mean-variance optimizer to individual assets such as stocks while using sample moments as inputs

  * Sample average are very bad estimators of a stock expected return going forward
  
> Why?

* Using for broad asset classes is slightly better as we expect the expected return of broad asset classes to be more stable over time, but still the minimizer is sensitive with respect to expected returns, which still have considerable uncertainty

> So what do people do?


* Impose restrictions on parameters, specially the ones that are poorly estimated (individual assets expected returns)

  * Minimum-variance investing
  
  * Risk parity
  

  
* Impose economics resticitions on parameters,i.e., equilibrium logic.

     * The Goldman Sachs/Black-Litterman approach

     * Factor investing




  



## Minimum-variance investing


* Mean-variance investing under the assumption that all assets have same expected returns, but uses covariance matrix to minimize risk

 * The idea is that expected returns signals are pure noise so we might as well assume there are no signals and all expected returns are the same
  
 * Can be ueful when applied within asset class when assumption of same expected returns is plausible
  
 * For example this is behind Minimum volatility investing [for example,https://investor.vanguard.com/mutual-funds/profile/VMVFX]
  
* Assumes all assets share the same expected return

$$Min~ W'Var(R)W~ subject~ to~ \mathbf{1}'W=1$$

$$W_{minvariance}=Var(R)^{-1}\mathbf{1}$$

* I use this technique in "hedging-risk factors", a recent academic paper, to construct portfolios with Sharpe-ratios of 0.8 at the yearly horizon, which is very large
> Why this might make sense?


In [2]:
Wmve

NameError: name 'Wmve' is not defined

In [ ]:
print(ERe.values)
er= (ERe.values*0+1)
print(er)

In [ ]:
# to find the minimum variance weights simply replace  the vector of expected returns by a vector of constant.

# in this case I am usign the average, but the value does not matter for the weights, why?
Cove=Re.cov()
er= (ERe.values*0+ERe.mean())
Wminv =np.linalg.inv(Cove) @ er
Wmve['minv']=Wminv/Wminv.sum()

# 

Wmve

>why these weights look like this ? Why are they shorting some assets?

In [ ]:
# what is the vol of the minimum-variance portfolio

(Wmve['minv'].T @ Cove @ Wmve['minv'])**0.5*(12**0.5)

In [ ]:
er

In [ ]:
# What is the expected return of the minimum variance portfolio?
# made up expected return estimate
# using the average across assets
print(er @ Wmve['minv']*12)|

# using each asset average return

print(ERe.values @ Wmve['minv']*12)

In [ ]:
# Sharpe ratio

# using the average across assets
print(((er @ Wmve['minv'])/(Wmve['minv'].T @ Cove @ Wmve['minv'])**0.5)*(12**0.5))

#using each asset average return

(ERe.values @ Wmve['minv'])/(Wmve['minv'].T @ Cove @ Wmve['minv'])**0.5*(12**0.5)

In [ ]:
# Compare with the Sharpe ratio of the tangency portfolio
(ERe.values @ Wmve['samplemean'])/(Wmve['samplemean'].T @ Cove @ Wmve['samplemean'])**0.5*(12**0.5)

In [ ]:
# compare minimum variance and tangency

# Training sample goes up to 1990
Re_estimate=Re['1960':'1990']

ER=Re_estimate.mean()
cov=Re_estimate.cov()
Wmve0=ER @ np.linalg.inv(cov)
Wmve0=Wmve/np.sum(Wmve)
print(Wmve)
#ER=1+0*Re_estimate.mean()
ER=np.ones(5)
cov=Re_estimate.cov()
Wmin=ER @ np.linalg.inv(cov)
Wmin=Wmin/np.sum(Wmin)
print(Wmin)

Re_test=Re['1991':]
ER=Re_test.mean()
cov=Re_test.cov()


### Moments in the TEST sample that starts in 1991
#average returns
print(Wmve @ ER) #average return of tangency in the TEST sample
print(Wmin @ ER) #average return of minimum variance in the TEST sample
#variance
print(Wmve @ cov @ Wmve) #variance return of tangency in the TEST sample
print(Wmin @ cov @ Wmin)#variance return of min variance in the TEST sample
# sharpe ratios
print(12**0.5*(Wmve @ ER)/(Wmve @ cov @ Wmve)**0.5)
print(12**0.5*(Wmin @ ER)/(Wmin @ cov @ Wmin)**0.5)

In [ ]:
ER=1+0*Re_estimate.mean()
ER=np.ones(5)
ER

* Which one is right? 

* The one with the average acorss assets or the ones using each asset average return?

* isn't using the average always right since it is the consistent with the data?

* Does it make sense to compare the in-sample tangency and the minimum-variance using the sample?

> Which one will always win?

## Risk- Parity investing


* Risk-parity investing: Mean-variance investing under the assumption that all expected returns are equal, and the correlations across all assets are zero
  * This is the strategy behind the biggest hedge funds on the planet [https://www.bridgewater.com/resources/our-thoughts-about-risk-parity-and-all-weather.pdf]
  

* It assumes that the non-diagonal terms in the covariacne matrix are all zero.

$$Var_{RP}(R)=diag(Var(R))=\left[\begin{array}{ccc}\sigma_1^2 & 0 & 0\\0 & \sigma_2^2 &0 \\0 & 0& \sigma^2_3\end{array}\right]$$

* we then have 

$$Min~ W'Var_{RP}(R)W ~ subject~ to~ \mathbf{1}'W=1$$

* it follows

$$W_{RP}=Var(R)^{-1}\mathbf{1}=\left[\begin{array}{c}\frac{1}{\sigma_1^2}\\\frac{1}{\sigma_2^2}\\\frac{1}{\sigma_3^2}\end{array}\right]$$

> Why this might make sense?

> Why does the assumption of ZERO correlation a conservative one? 

> Doesn't this underestimate the degree of comovement across stocks?

In [ ]:
Covrp=Cove.copy()
print(Covrp)
print(np.diag(Covrp))
Covrp=np.diag(np.diag(Covrp))
Covrp

In [ ]:
er

In [ ]:

Wrp =np.linalg.inv(Covrp) @ er
Wmve['rp']=Wrp/Wrp.sum()
Wmve

In [ ]:
# what is the vol of the risk portfolio, using the diagonal assumption
#doesn' make any sense as a way to evaluate the strategy!!!!

print((Wmve['rp'].T @ Covrp @ Wmve['rp'])**0.5)


# using the actual covariance matrix

(Wmve['rp'].T @ Cove @ Wmve['rp'])**0.5

In [ ]:
# What is the expected return of the minimum variance portfolio?

# using the average across assets
print(er @ Wmve['rp'])

# using each asset average return

ERe.values @ Wmve['rp']

In [ ]:
# Sharpe ratio

# using the average across assets, and diagnonal assumption
print((er @ Wmve['rp'])/(Wmve['rp'].T @ Covrp @ Wmve['rp'])**0.5)

# using each asset average return, actual covariacne matrix estimated from the data

(ERe.values @ Wmve['rp'])/(Wmve['rp'].T @ Cove @ Wmve['rp'])**0.5

* Which one is right? 

* The one that uses the assumptions or the ones that we actually find in the data?

In [ ]:
Wmvec=Wmve.copy()
Wmvec.loc['MKTUS','rp']=-3
print(Wmvec['rp'])
np.abs(Wmvec['rp'])/np.abs(Wmvec['rp'].sum())

In [ ]:
# compare minimum variance and tangency

# Training sample goes up to 1990
Re_estimate=Re['1960':'1980']

ER=Re_estimate.mean()
cov=Re_estimate.cov()
Wmve0=ER @ np.linalg.inv(cov)
Wmve0=Wmve0/np.sum(Wmve0)
print(Wmve)
#ER=1+0*Re_estimate.mean()
ER=np.ones(5)
cov=np.diag(np.diag(Re_estimate.cov()))
Wmin=ER @ np.linalg.inv(cov)
Wmin=Wmin/np.sum(Wmin)
print(Wmin)

Re_test=Re['1981':]
ER=Re_test.mean()
cov=Re_test.cov()


### Moments in the TEST sample that starts in 1991
#average returns
print(Wmve0 @ ER) #average return of tangency in the TEST sample
print(Wmin @ ER) #average return of minimum variance in the TEST sample
#variance
print(Wmve0 @ cov @ Wmve0) #variance return of tangency in the TEST sample
print(Wmin @ cov @ Wmin)#variance return of min variance in the TEST sample
# sharpe ratios
print(12**0.5*(Wmve0 @ ER)/(Wmve0 @ cov @ Wmve0)**0.5)
print(12**0.5*(Wmin @ ER)/(Wmin @ cov @ Wmin)**0.5)

# LEcture 11/24 ended here

Should this move somewhere else?aa

# Equilibrium thinking as a response to estimation uncertainty

* Another way that people response to the estimation uncertainty problem is to impose economic restrictions
* Not everyone can invest in the higher expected return assets
* If we were all to invest in the higher expected return assets, prices would go up and expected returns would go down
* Imposing equilibrium– that is:
  * Assume all investors in the world are choosing optimal portfolios, 
  * Then solve for the expected returns that make all the assets in the world be held
  * This is imposing equilibrium: since all assets must be held, and investors must want to hold them
  * Then expected returns have to adjust so everyone is happy and all assets are held
      
